In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['FPS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [6]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [7]:
list(zip(longforms, counts))

[('fear potentiated startle', 84),
 ('farnesyl diphosphate synthase', 14),
 ('polysaccharide from fuzi', 6),
 ('self reported fat preference', 5),
 ('fat preference score', 4),
 ('family protection service', 4),
 ('formyl peptide sensor', 4),
 ('farnesyl pyrophosphate synthase', 4),
 ('fpp synthase', 3),
 ('progression free survival', 3),
 ('fractional photothermolysis systemic', 2),
 ('frames per second', 2),
 ('functional performance status', 2),
 ('filarial parasitic sheath protein', 2),
 ('f prostanoid receptor', 2)]

In [12]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [13]:
result = [grounding_map, names, pos_labels]

In [14]:
result

[{'f prostanoid receptor': 'HGNC:9600',
  'family protection service': 'ungrounded',
  'farnesyl diphosphate synthase': 'HGNC:3631',
  'farnesyl pyrophosphate synthase': 'HGNC:3631',
  'fat preference score': 'ungrounded',
  'fear potentiated startle': 'MESH:D013216',
  'filarial parasitic sheath protein': 'UP:Q25402',
  'formyl peptide sensor': 'FPLX:FPR',
  'fpp synthase': 'HGNC:3631',
  'fractional photothermolysis systemic': 'ungrounded',
  'frames per second': 'ungrounded',
  'functional performance status': 'ungrounded',
  'polysaccharide from fuzi': 'ungrounded',
  'progression free survival': 'ungrounded',
  'self reported fat preference': 'ungrounded'},
 {'HGNC:9600': 'PTGFR',
  'HGNC:3631': 'FDPS',
  'MESH:D013216': 'Reflex, Startle',
  'UP:Q25402': 'Microfilarial sheath protein SHP3',
  'FPLX:FPR': 'FPR'},
 ['FPLX:FPR', 'HGNC:3631', 'HGNC:9600', 'MESH:D013216', 'UP:Q25402']]

In [15]:
grounding_map, names, pos_labels = [{'f prostanoid receptor': 'HGNC:9600',
  'family protection service': 'ungrounded',
  'farnesyl diphosphate synthase': 'HGNC:3631',
  'farnesyl pyrophosphate synthase': 'HGNC:3631',
  'fat preference score': 'ungrounded',
  'fear potentiated startle': 'MESH:D013216',
  'filarial parasitic sheath protein': 'UP:Q25402',
  'formyl peptide sensor': 'FPLX:FPR',
  'fpp synthase': 'HGNC:3631',
  'fractional photothermolysis systemic': 'ungrounded',
  'frames per second': 'ungrounded',
  'functional performance status': 'ungrounded',
  'polysaccharide from fuzi': 'ungrounded',
  'progression free survival': 'ungrounded',
  'self reported fat preference': 'ungrounded'},
 {'HGNC:9600': 'PTGFR',
  'HGNC:3631': 'FDPS',
  'MESH:D013216': 'Reflex, Startle',
  'UP:Q25402': 'Microfilarial sheath protein SHP3',
  'FPLX:FPR': 'FPR'},
 ['FPLX:FPR', 'HGNC:3631', 'HGNC:9600', 'MESH:D013216', 'UP:Q25402']]

In [16]:
excluded_longforms = []

In [17]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [18]:
additional_entities = {}

In [19]:
unambiguous_agent_texts = {}

In [20]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [17]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [18]:
names.update({key: value[0] for key, value in additional_entities.values()})
pos_labels = list(set(pos_labels) | additional_entities.keys())

In [21]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 02:24:04] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 02:24:07] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9240364828408307 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [22]:
classifier.stats

{'label_distribution': {'UP:Q25402': 2,
  'MESH:D013216': 54,
  'HGNC:3631': 17,
  'HGNC:9600': 1,
  'ungrounded': 11,
  'FPLX:FPR': 1},
 'f1': {'mean': 0.924036, 'std': 0.050174},
 'precision': {'mean': 0.895452, 'std': 0.066294},
 'recall': {'mean': 0.960833, 'std': 0.032016},
 'HGNC:9600': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'UP:Q25402': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'MESH:D013216': {'f1': {'mean': 0.94776, 'std': 0.032694},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.902564, 'std': 0.059777}},
 'ungrounded': {'f1': {'mean': 0.7, 'std': 0.163299},
  'pr': {'mean': 0.566667, 'std': 0.226078},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'FPLX:FPR': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:3631': {'f1': {'mean': 0.942857, 'std': 0.069985},
  'pr': {'mean'

In [23]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [24]:
disamb.dump(model_name, results_path)

In [25]:
print(disamb.info())

Disambiguation model for FPS

Produces the disambiguations:
	FDPS*	HGNC:3631
	FPR*	FPLX:FPR
	Microfilarial sheath protein SHP3*	UP:Q25402
	PTGFR*	HGNC:9600
	Reflex, Startle*	MESH:D013216

Class level metrics:
--------------------
Grounding                        	Count	F1     
                  Reflex, Startle*	54	0.94776
                             FDPS*	17	0.94286
                       Ungrounded	11	    0.7
Microfilarial sheath protein SHP3*	 2	    0.4
                            PTGFR*	 1	    0.0
                              FPR*	 1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.92404
	Precision:	0.89545
	Recall:		0.96083

* Positive labels
See Docstring for explanation



In [26]:
model_to_s3(disamb)